In [1]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
from src.AIGuardian.Tasks.KafkaAgent import KafkaAgent

agent = KafkaAgent(end_processing=20)
agent.receive_messages()

Agent Loop Count: 0 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 1 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 2 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 3 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 4 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 5 and last consumed time: 2025-05-21 21:20:54.346055
Agent Loop Count: 6 and last consumed time: 2025-05-21 21:20:54.346055
==> Reviewing waiting tasks
Agent Loop Count: 7 and last consumed time: 2025-05-21 21:21:05.734881
==> Running DataColumnType with prompt: None
==> Task 67d7a13f-4e77-43b5-ba16-3e9519278a2e complete_task
==> Running DataColumnRefiner with prompt: None
==> Task 8b16e64a-061b-43a3-8f84-d012449d15ac complete_task
==> Reviewing waiting tasks
==> Removing 54f7eee4-a370-4b5c-8800-8daa9ada68a1 from child tasks.
==> Removing 67d7a13f-4e77-43b5-ba16-3e9519278a2e from child tasks.
==> Removing b1ff4c8d-e304-

KeyboardInterrupt: 

In [3]:
print(agent.waiting_tasks)
print(agent.processed_tasks)

[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001AACD08BD90>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001AACE508190>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001AACE5459D0>]
[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001AACE3FAC90>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001AACD08BD90>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001AACD08B550>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001AACD0CFC50>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001AACE4B71D0>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001AACE4EFF50>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001AACE4EFFD0>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001AACE4F8390>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A

---
## Debug the Agent not fully waiting

In [4]:
from src.MetaFort.AILoggingTopics import AILoggingTopics
print(agent.waiting_tasks[0].child_task)
print(agent.db_engine.consumers[AILoggingTopics.AI_TASK_COMPLETED_TOPIC].config['group_id'])
print(agent.waiting_tasks[0].is_waiting())


['98c90e2b-9fe7-4d77-a502-fbd884215758', '5e1cde37-d80a-40c7-8377-a76364a2f105']
task-completed-029fa456-7afc-439e-b9ac-485d1dd2c884
True


In [ ]:
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine

print("==> Reviewing waiting tasks")
for task in agent.waiting_tasks:
    print(f"==> Reviewing task {task.task_id}")
    print(task.is_waiting())

==> Reviewing waiting tasks


In [ ]:
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine
kafka_engine = KafkaEngine('localhost:9092')

print("==> Reviewing waiting tasks")
for task in agent.waiting_tasks:
    print(f"==> Reviewing task {task.task_id}")
    for child_task in task.child_task:
        print(f"=====> Reviewing child task {child_task}")
        found_msg = kafka_engine.search_batch_topic(AILoggingTopics.AI_TASK_COMPLETED_TOPIC, search_value=child_task, search_key='task_id')
        if found_msg:
            print(f"=====> Found message for child task {child_task}")
            task.child_task.remove(child_task)
    if not task.is_waiting():
        print('BOOM!!!')
        agent.waiting_tasks.remove(task)

==> Reviewing waiting tasks
==> Reviewing task 91a55f45-0853-43e3-8da2-f235fea62a21
=====> Reviewing child task ca0ce828-b739-44ca-b90b-7e3708e6e655
Searching topic 'ai_tasks_completed' for value: ca0ce828-b739-44ca-b90b-7e3708e6e655
This will timeout after 60 seconds if no more messages are available.
Processing 129 messages from partition TopicPartition(topic='ai_tasks_completed', partition=2)
Processing 139 messages from partition TopicPartition(topic='ai_tasks_completed', partition=1)
Processing 127 messages from partition TopicPartition(topic='ai_tasks_completed', partition=3)
Processing 126 messages from partition TopicPartition(topic='ai_tasks_completed', partition=0)
Found match at offset 124 in partition 0
Processing 154 messages from partition TopicPartition(topic='ai_tasks_completed', partition=4)
Search completed in 1.13 seconds.
Processed 675 messages, found 1 matches.
=====> Found message for child task ca0ce828-b739-44ca-b90b-7e3708e6e655
BOOM!!!
